In [1]:
import json
import pymbe.api as pm

from pathlib import Path

from typing import Any, Collection, Dict, List, Tuple, Union

from pymbe.model import Model, Element
from pymbe.model_modification import build_from_classifier_pattern, \
                                    new_element_ownership_pattern, \
                                    build_from_binary_relationship_pattern, \
                                    build_unioning_superset_classifier, \
                                    build_from_feature_pattern, \
                                    build_from_binary_assoc_pattern, \
                                    build_from_binary_connector_pattern, \
                                    apply_covered_feature_pattern

from pymbe.query.metamodel_navigator import is_type_undefined_mult, \
                                    is_multiplicity_one, \
                                    is_multiplicity_specific_finite, \
                                    get_finite_multiplicity_types, \
                                    identify_connectors_one_side, \
                                    get_lower_multiplicty, \
                                    get_upper_multiplicty

from uuid import uuid4

In [2]:
filename = "TrashCollection"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "example_data" / filename

boat_data = pm.Model.load_from_post_file(json_file)
boat_data

c:\users\bjorn cole\documents\github\pymbe\src\pymbe\label.py:115: UserWarning: Cannot process Expression elements yet!
  warn(f"Cannot process {expression._metatype} elements yet!")


<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\dev_docs\data_loading\example_data\TrashCollection.json)>

In [3]:
packages = [ele for ele in boat_data.elements.values() if ele._metatype == 'Package']
packages

[Trash Collections «Package»]

In [4]:
analyses = \
    [ele for ele in packages[0].throughOwningMembership if ele._metatype == 'AnalysisCaseDefinition']
analyses

[Trash Collection Efficiency Analysis «AnalysisCaseDefinition»]

In [5]:
analyses[0].throughSubjectMembership[0]

Trash Removal Fleet: Trash Removal Fleet «ReferenceUsage»

In [6]:
fleet = analyses[0].throughSubjectMembership[0].throughFeatureTyping[0]
fleet

Trash Removal Fleet «PartDefinition»

In [7]:
surface_skimmer = fleet.throughFeatureMembership[0].throughFeatureTyping[0].reverseSubclassification[0]
fleet.throughFeatureMembership[0].throughFeatureTyping[0].reverseSubclassification

[Water Surface Trash Skimmer «PartDefinition»,
 Net-Based Skimmer «PartDefinition»,
 Trash Processor Mothership «PartDefinition»]

In [8]:
surface_skimmer.throughFeatureMembership

[Skim Trash: Skim Trash Def «PerformActionUsage»]

In [9]:
build_from_feature_pattern(
    owner=surface_skimmer,
    name='width',
    model=boat_data,
    specific_fields = {},
    feature_type=None,
    direction="",
    metatype="AttributeUsage",
    connector_end = False,
)

width «AttributeUsage»

In [10]:
surface_skimmer.throughFeatureMembership

[Skim Trash: Skim Trash Def «PerformActionUsage», width «AttributeUsage»]